In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import requests
from bs4 import BeautifulSoup
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score

In [2]:
dfs = []

# Loop through years from 2021 to 2023
for year in range(2021, 2024):
    filename = f"ATP_Matches/atp_matches_{year}.csv"
    df = pd.read_csv(filename)
    dfs.append(df)

dfs

[     tourney_id    tourney_name surface  draw_size tourney_level  \
 0     2021-0096  Tokyo Olympics    Hard         64             A   
 1     2021-0096  Tokyo Olympics    Hard         64             A   
 2     2021-0096  Tokyo Olympics    Hard         64             A   
 3     2021-0096  Tokyo Olympics    Hard         64             A   
 4     2021-0096  Tokyo Olympics    Hard         64             A   
 ...         ...             ...     ...        ...           ...   
 2728  2021-9210       Laver Cup    Hard          8             A   
 2729  2021-9210       Laver Cup    Hard          8             A   
 2730  2021-9210       Laver Cup    Hard          8             A   
 2731  2021-9210       Laver Cup    Hard          8             A   
 2732  2021-9210       Laver Cup    Hard          8             A   
 
       tourney_date  match_num  winner_id  winner_seed winner_entry  ...  \
 0         20210724        237     126207          NaN          NaN  ...   
 1         2021072

In [3]:
# Load the merged DataFrame
merged_df = pd.concat(dfs, ignore_index=True)
# merged_df = merged_df.dropna()
merged_df

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2021-0096,Tokyo Olympics,Hard,64,A,20210724,237,126207,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,53.0,1228.0,71.0,996.0
1,2021-0096,Tokyo Olympics,Hard,64,A,20210724,238,105526,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,48.0,1410.0,95.0,829.0
2,2021-0096,Tokyo Olympics,Hard,64,A,20210724,239,111576,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,160.0,471.0,197.0,354.0
3,2021-0096,Tokyo Olympics,Hard,64,A,20210724,240,105357,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,44.0,1476.0,61.0,1106.0
4,2021-0096,Tokyo Olympics,Hard,64,A,20210724,241,207830,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,145.0,525.0,137.0,570.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8631,2023-M-DC-2023-WG2-PO-RSA-LUX-01,Davis Cup WG2 PO: RSA vs LUX,NaN,4,D,20230204,5,202335,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1717.0,1.0
8632,2023-M-DC-2023-WG2-PO-TUN-CYP-01,Davis Cup WG2 PO: TUN vs CYP,NaN,4,D,20230203,1,117365,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,990.0,11.0,279.0,190.0
8633,2023-M-DC-2023-WG2-PO-TUN-CYP-01,Davis Cup WG2 PO: TUN vs CYP,NaN,4,D,20230203,2,121411,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,364.0,131.0,894.0,15.0
8634,2023-M-DC-2023-WG2-PO-TUN-CYP-01,Davis Cup WG2 PO: TUN vs CYP,NaN,4,D,20230203,4,144949,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,894.0,15.0,285.0,184.0


In [4]:
# Dictionary of all players
player_profiles = {}

for index, row in merged_df.iterrows():
    winner_id = row['winner_id']
    loser_id = row['loser_id']
    match_details = {
        'tournament': row['tourney_name'],
        'surface': row['surface'],
        'score': row['score'],
        'minutes': row['minutes'],
        'winner': winner_id,
        'loser': loser_id
        # Add more match details as needed
    }

    total_w_aces = row['w_ace'] if not pd.isnull(row['w_ace']) else 0
    total_w_df = row['w_df'] if not pd.isnull(row['w_df']) else 0
    total_l_aces = row['l_ace'] if not pd.isnull(row['l_ace']) else 0
    total_l_df = row['l_df'] if not pd.isnull(row['l_df']) else 0
    row_Minutes = row['minutes'] if not pd.isnull(row['minutes']) else 0

    if winner_id in player_profiles:
        player_profiles[winner_id]['matches'].append(match_details)
        player_profiles[winner_id]['total_w_aces'] += total_w_aces
        player_profiles[winner_id]['total_w_df'] += total_w_df
        player_profiles[winner_id]['num_of_wins'] += 1
        player_profiles[winner_id]['total_minutes'] += row_Minutes
    else:
        player_profiles[winner_id] = {
            'name': row['winner_name'],
            'hand': 1 if row['winner_hand'] == 'R' else 0,
            'height': row['winner_ht'],
            'country': row['winner_ioc'],
            'age': row['winner_age'],
            'matches': [match_details],
            'total_w_aces': total_w_aces,
            'total_w_df': total_w_df,
            'total_l_aces': 0,
            'total_l_df': 0,
            'rank': row['winner_rank'],
            'rank_points': row['winner_rank_points'],
            'num_of_wins': 1, 
            'num_of_loss': 0,
            'total_minutes': row_Minutes
        }

    if loser_id in player_profiles:
        player_profiles[loser_id]['matches'].append(match_details)
        player_profiles[loser_id]['total_l_aces'] += total_l_aces
        player_profiles[loser_id]['total_l_df'] += total_l_df
        player_profiles[loser_id]['num_of_loss'] += 1
        player_profiles[loser_id]['total_minutes'] += row_Minutes
    else:
        player_profiles[loser_id] = {
            'name': row['loser_name'],
            'hand': 1 if row['loser_hand'] == 'R' else 0,
            'height': row['loser_ht'],
            'country': row['loser_ioc'],
            'age': row['loser_age'],
            'matches': [match_details],
            'total_w_aces': 0,
            'total_w_df': 0,
            'total_l_aces': total_l_aces,
            'total_l_df': total_l_df,
            'rank': row['loser_rank'],
            'rank_points': row['loser_rank_points'],
            'num_of_wins': 0, 
            'num_of_loss': 1,
            'total_minutes': row_Minutes 
        }

first_5_players = list(player_profiles.items())[:5]

for player_id, profile in list(player_profiles.items()):
    print(profile['name'])

Frances Tiafoe
Soon Woo Kwon
Jan Lennard Struff
Thiago Monteiro
Sumit Nagal
Denis Istomin
John Millman
Lorenzo Musetti
Tomas Machac
Joao Sousa
Mikhail Kukushkin
Federico Coria
Dominik Koepfer
Facundo Bagnis
Miomir Kecmanovic
Kamil Majchrzak
Marcos Giron
Norbert Gombos
Egor Gerasimov
Gilles Simon
Daniel Elahi Galan
Mohamed Safwat
James Duckworth
Lukas Klein
Marin Cilic
Joao Menezes
Jeremy Chardy
Tomas Barrios Vera
Liam Broady
Francisco Cerundolo
Nikoloz Basilashvili
Roberto Carballes Baena
Alejandro Davidovich Fokina
Pedro Sousa
Fabio Fognini
Yuichi Sugita
Ugo Humbert
Pablo Andujar
Lorenzo Sonego
Taro Daniel
Karen Khachanov
Yoshihito Nishioka
Aslan Karatsev
Tommy Paul
Ilya Ivashka
Gael Monfils
Max Purcell
Felix Auger Aliassime
Diego Schwartzman
Juan Pablo Varillas
Hubert Hurkacz
Luke Saville
Pablo Carreno Busta
Tennys Sandgren
Kei Nishikori
Andrey Rublev
Alexander Zverev
Yen Hsun Lu
Stefanos Tsitsipas
Philipp Kohlschreiber
Daniil Medvedev
Alexander Bublik
Novak Djokovic
Hugo Dellien
Pab

In [5]:
# def get_player_ranking(player_name):
#     url = 'https://www.atptour.com/en/rankings/singles'
    
#     # Send a GET request to the ATP rankings page
#     response = requests.get(url)
    
#     # Check if the request was successful
#     if response.status_code == 200:
#         # Parse the HTML content of the page
#         soup = BeautifulSoup(response.content, 'html.parser')
        
#         # Find the table containing the rankings
#         rankings_table = soup.find('table', class_='mobile-table mega-table non-live')
        
#         # Check if the rankings table was found
#         if rankings_table:
#             # Find all rows in the table (skipping the header row)
#             rows = rankings_table.find_all('tr')[1:]
            
#             # Iterate over each row
#             for row in rows:
#                 # Extract player name and ranking from the row
#                 player_name_atp = row.find('td', class_='player-cell').find('a').text.strip()
#                 player_rank = row.find('td', class_='rank-cell').text.strip()
                
#                 # Check if the player name matches
#                 if player_name_atp == player_name:
#                     return player_rank
            
#             # If the player's name is not found in the rankings
#             return "Not Ranked"
#         else:
#             print("Error: Unable to find rankings table.")
#             return None
    
#     else:
#         print('Failed to retrieve ATP rankings:', response.status_code)
#         return None


# for player_id, profile in player_profiles.items():
#     player_name = profile['name']
#     print(player_name)
    
#     ranking = get_player_ranking(player_name)
    
#     profile['rank'] = ranking

# for player_id, profile in list(player_profiles.items())[:5]:
#     print(f"Player Name: {profile['name']}")
#     print(f"ATP Ranking: {profile['rank']}")
#     print()

def get_player_ranking(player_name):
    url = 'https://www.atptour.com/en/rankings/singles?RankRange=0-5000&Region=all&DateWeek=Current%20Week'
    
    response = requests.get(url)
    
    if response.status_code == 200:
        
        soup = BeautifulSoup(response.text, 'html.parser') 
        
        rankings_table = soup.find('table', class_='mobile-table mega-table non-live')
        
        if rankings_table:
            
            rows = rankings_table.find_all('tr')[1:]
            
            for row in rows:
                last_name = row.find('span', class_='lastName')
                if last_name is not None:
                    player_name_atp = last_name.text.strip()
                    rank = row.find('td', class_='rank bold heavy tiny-cell')
                    if rank is not None:
                        player_rank = rank.text.strip()
                        #print(player_name_atp, ':', player_rank)
                
                if player_name_atp == player_name:
                    return player_rank
            
            return "Not Ranked"
        else:
            print("Error: Unable to find rankings table.")
            return None
    
    else:
        print('Failed to retrieve ATP rankings:', response.status_code)
        return None

#print(get_player_ranking("A. Kim"))
"""for player_element, element_attributes in list(player_profiles.items()): 
    player1_name_array = element_attributes['name'].split()
    #print("Player 1 name", player1_name)
    player1_name_array[0] = player1_name_array[0][0]
    #print("Player 1 Array ", player1_name_array)
    player1_name = player1_name_array[0] + "." + " " + player1_name_array[1]
    player1_rank = get_player_ranking(player1_name)
    print(player1_rank)"""

'for player_element, element_attributes in list(player_profiles.items()): \n    player1_name_array = element_attributes[\'name\'].split()\n    #print("Player 1 name", player1_name)\n    player1_name_array[0] = player1_name_array[0][0]\n    #print("Player 1 Array ", player1_name_array)\n    player1_name = player1_name_array[0] + "." + " " + player1_name_array[1]\n    player1_rank = get_player_ranking(player1_name)\n    print(player1_rank)'

In [6]:
X = []
Y = []

def bmi_index(weight, height):
    if height == 0:
        return 0
    else:
        return weight / (height ** 2)

for player_id, attributes in list(player_profiles.items()):
    matchesList = attributes['matches']
    for match in matchesList:
        x_components = []
        
        if match['winner'] == player_id:
            player2_id = match['loser']
        else:
            player2_id = match['winner']
        
        player2_profile = player_profiles[player2_id]
            
        if np.isnan(attributes['hand']) or np.isnan(attributes['height']) or np.isnan(attributes['age']) or np.isnan(attributes['rank']) or np.isnan(player2_profile['hand']) or np.isnan(player2_profile['height']) or np.isnan(player2_profile['age']) or np.isnan(player2_profile['rank']):
            continue
        else:
            # Player 1     
            """player1_name_array = attributes['name'].split()
            #print("Player 1 name", player1_name)
            player1_name_array[0] = player1_name_array[0][0]
            #print("Player 1 Array ", player1_name_array)
            player1_name = player1_name_array[0] + "." + " " + player1_name_array[1]
            player1_rank = get_player_ranking(player1_name)"""
            
            x_components.append(attributes['hand'])
            x_components.append(bmi_index(attributes['age'], attributes['height'] / 100))
            x_components.append(attributes['total_w_aces']  + attributes['total_l_aces'])
            x_components.append(attributes['rank'])
            x_components.append(attributes['rank_points'])
            x_components.append(attributes['num_of_wins'] / (attributes['num_of_wins'] + attributes['num_of_loss']) * 100)
        
            # Player 2
            """player2_name_array = player2_profile['name'].split()
            #print("Player 1 name", player1_name)
            player2_name_array[0] = player2_name_array[0][0]
            #print("Player 2 Array ", player2_name_array)
            player2_name = player2_name_array[0] + "." + " " + player2_name_array[1]
            player2_rank = get_player_ranking(player2_name)"""
            
            x_components.append(player2_profile['hand'])
            x_components.append(bmi_index(player2_profile['age'], player2_profile['height'] / 100))
            x_components.append(player2_profile['total_w_aces'] + player2_profile['total_l_aces'])
            x_components.append(player2_profile['rank'])
            x_components.append(player2_profile['rank_points'])
            x_components.append(player2_profile['num_of_wins'] / (player2_profile['num_of_wins'] + player2_profile['num_of_loss']) * 100)
        
            x_components.append(attributes['rank'] - player2_profile['rank'])
            x_components.append(attributes['rank_points'] - player2_profile['rank_points'])

            if (attributes['total_w_df'] + attributes['total_l_df']) > (player2_profile['total_w_df'] + player2_profile['total_l_df']):
                x_components.append(1)
            elif (attributes['total_w_df'] + attributes['total_l_df']) < (player2_profile['total_w_df'] + player2_profile['total_l_df']):
                x_components.append(-1)
            else:
                x_components.append(0)
            
            
            X.append(x_components)
  
            if match['winner'] == player_id:
                Y.append(1)
            else:
                Y.append(2)
    
    
X = np.array(X)
Y = np.array(Y)

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=True)

In [8]:
logModel = LogisticRegression(max_iter=100, random_state=42)
logModel.fit(X_train, Y_train)

C:\Users\prane\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [9]:
Y_pred = logModel.predict(X_test)
Y_pred

array([2, 2, 1, ..., 1, 1, 2])

In [10]:
accuracyScore = accuracy_score(Y_test, Y_pred)
accuracyScore * 100

69.26242481798037